Anthropic supports both sync and async! This is great because we can wait for functions to finish before we use them! 

In this example, we will make a program called "Titan Support Protocol." In this example, we will assign our mech a personality type and have a message generated based on our Titan's health (Which we randomly choose). We also send four generated UUIDs which are generated while the LLM runs

First, we start by importing Agentops and Anthropic

In [ ]:
!pip install agentops
!pip install anthropic

Setup our generic default statements

In [ ]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
import random #We don't need this for agentops, we use this to generate a message later
import asyncio #We don't need this for agentops, we use this to run both async tasks and await them both finishing later
import uuid #We don't need this for agentops, we use this to generate UUIDs
from dotenv import load_dotenv

And set our API keys.

In [ ]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "<your_anthropic_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"


Now let's set the client as Anthropic

In [ ]:
client = Anthropic()

Now we create three personality presets; 

Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower, Northstar is a precise and agile sniper that excels in long-range combat and flight, while Ronin is a swift and aggressive melee specialist who thrives on close-quarters hit-and-run tactics.

In [ ]:
TitanPersonality = [
    "Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower. He speaks bluntly.,", 
    "Northstar is a precise and agile sniper that excels in long-range combat and flight. He speaks with an edge of coolness to him", 
    "Ronin is a swift and aggressive melee specialist who thrives on close-quarters hit-and-run tactics. He talks like a Samurai might."
]

And our comabt log generator! We select from four health presets!

In [ ]:
TitanHealth = [
    "Fully functional", "Slightly Damaged", "Moderate Damage", "Considerable Damage", "Near Destruction"
]

Now to the real core of this; making our message stream! We create this as a function we can call later! I personally made user (Sends an initialization prompt), computer (sends the titan personality and health) and assistant (The LLM itself). I also create examples since the LLM's context size can handle it!

In [ ]:
async def req() -> None:
    message = await client.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "You are a Titan; a mech from Titanfall 2. Based on your titan's personality and status, generate a message for your pilot. If Near Destruction, make an all caps death message such as AVENGE ME or UNTIL NEXT TIME."
            },
            {
                "role": "computer",
                "content": "Personality: Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower. He speaks bluntly. Status: Considerable Damage"
            },
            {
                "role": "assistant",
                "content": "Heavy damage detected. Reinforcements would be appreciated, but I can still fight."
            },
            {
                "role": "user",
                "content": "You are a Titan; a mech from Titanfall 2. Based on your titan's personality and status, generate a message for your pilot. If Near Destruction, make an all caps death message such as AVENGE ME or UNTIL NEXT TIME."
            },
            {
                "role": "computer",
                "content": f"Personality: {random.choice(TitanPersonality)}. Status: {random.choice(TitanHealth)}"
            }
        ],
        model="claude-3-5-sonnet-20240620",
    )
    llmgentext = (message.content[0].text)
    
    
uuids = []
    
async def generate_uuids():
    # Generate 4 UUIDs and print them vertically while we await the LLM's finish
    uuids = [str(uuid.uuid4()) for _ in range(4)]
    
    
    

Now we wrap it all in a nice main function!Run this for the magic to happen! Go to your AgentOps dashboard and you should see this session reflected!


In [ ]:
# Start both at the same time and wait
await asyncio.gather(generate_uuids(), req())
    
print("Combat log incoming from encrypted area ")
for u in uuids:
        print(u)

print(". Log Reads:", llmgentext)

agentops.end_session("Success") 

Run this for the magic to happen! Go to your AgentOps dashboard and you should see this session reflected!